<h1 style="color: darkblue; "> HIV Comorbidity in TBSim <h1>

This notebook demonstrates how to use the **HIV comorbidity module** in the `tbsim` package.

**Module Location:** `tbsim/comorbidities/hiv/`

---

## Overview

The HIV comorbidity module in TBSim allows users to simulate the impact of **HIV infection on TB disease dynamics**. Rather than modeling HIV progression in detail, it serves as a **lightweight comorbidity** that modifies TB risks based on HIV state.

The module is composed of three key components:

- **HIV Model**: Represents HIV infection as a comorbidity.
- **HIV Intervention**: Enables scenario testing via prevalence and ART coverage.
- **TB-HIV Connector**: Bridges HIV and TB models, applying comorbidity risk logic.

---

## HIV Model

The **HIV model** simulates HIV infection status and its progression through simplified states:

- `ACUTE`
- `LATENT`
- `AIDS`

This implementation is intended as a modifier for TB outcomes, rather than a standalone disease model.

### Key Parameters

| Parameter              | Description                                                                 |
|------------------------|-----------------------------------------------------------------------------|
| `init_prev`            | Probability of being HIV-positive (ACUTE stage) at simulation start         |
| `init_onart`           | Probability of being on ART if HIV-positive                                 |
| `ACUTE_to_LATENT`      | Daily probability of transitioning from ACUTE to LATENT                     |
| `LATENT_to_AIDS`       | Daily probability of transitioning from LATENT to AIDS                      |
| `AIDS_to_DEAD`         | Daily probability of death after reaching AIDS (currently unused)           |
| `art_progression_factor` | Multiplier reducing HIV progression for agents on ART                      |

---

## HIV Intervention

The `HivIntervention` class allows control over **HIV prevalence and ART coverage** in the population. This is useful for scenario testing and simulating public health campaigns.

### Supported Modes

- `'infection'`: Adjust HIV infection prevalence
- `'art'`: Adjust ART coverage
- `'both'`: Adjust both infection and treatment

### What It Does

- Applies changes to the **HIV state** of the population
- Operates within a specified time range (`start` to `stop`)
- Can be targeted to specific age groups or entire population

```python
intervention = HivInterventions(pars=dict(
    mode='both',
    prevalence=0.2,
    percent_on_ART=0.5,
    minimum_age=15,
    max_age=49,
    start=ss.date('2000-01-01'),
    stop=ss.date('2010-12-31'),
))
```

Also, the parameters shown above are the same ones that you can modify in order to run a different scenario. 

---

## TB-HIV Connector

The `TB_HIV_Connector` class links the TB and HIV models, simulating the **comorbidity effect** of HIV on TB progression.

### Purpose

Modifies TB progression parameters based on an individual's **current HIV state**.

### Risk Multiplier Table

| HIV State | TB Progression Risk Multiplier |
|-----------|-------------------------------|
| `ACUTE`   | 1.5                           |
| `LATENT`  | 2.0                           |
| `AIDS`    | 3.0                           |

### Behavior at Each Time Step

1. Identify all **TB-infected** individuals.
2. Look up their current **HIV state**.
3. Apply a **multiplier** to their TB `rr_activation` value (risk of progressing to active TB).

---

## Process Flow (Suggested Diagram)

```text
+-----------------+
|   TB Infection  |
+-----------------+
        |
        v
+-----------------+     +-----------------------+
|  HIV Comorbidity| --> | HIV State (ACUTE,...) |
+-----------------+     +-----------------------+
        |                          |
        |      TB_HIV_Connector    |
        +-------------------------> Applies RR Multiplier
                                   |
                                   v
                    +-------------------------------+
                    | TB rr_activation Adjusted     |
                    | Based on HIV State            |
                    +-------------------------------+
```

---

## Example Code

```python
import tbsim as mtb
import starsim as ss
import matplotlib.pyplot as plt

# Create the simulation
sim = ss.Sim(
    people=ss.People(n_agents=1000),
    diseases=[
        mtb.TB(pars=dict(init_prev=ss.bernoulli(p=0.25))),
        mtb.HIV(pars=dict(init_prev=ss.bernoulli(p=0.10)))
    ],
    connectors=[mtb.TB_HIV_Connector()],
    interventions=[
        mtb.HivInterventions(pars=dict(
            mode='both',
            prevalence=0.2,
            percent_on_ART=0.5,
            minimum_age=15,
            max_age=49,
        ))
    ],
    networks=ss.RandomNet(pars=dict(n_contacts=ss.poisson(lam=3), dur=0)),
    pars=dict(start=ss.date('1980-01-01'), stop=ss.date('2035-12-31')),
)

# Run and plot
sim.run()
sim.plot()
plt.show()
```

---



<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=d60e3a4a-7c07-4686-88c7-ebc3c2fbb3c7' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>